In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.special as special
from scipy.optimize import curve_fit
import seaborn as sns
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [3]:
df = pd.read_csv('Valores_Nulos_Limpios_Copehnague.csv')
#df.head(5)

**DATAFRAME CON VARIABLES DEPENDIENTES LIMPIAS**

In [4]:
dfD = df[['host_is_superhost', 'host_has_profile_pic', 'host_identity_verified', 'has_availability', 'instant_bookable', 'price', 'property_type', 'accommodates', 'room_type', 'review_scores_rating']]
dfD.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21528 entries, 0 to 21527
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   host_is_superhost       21528 non-null  object 
 1   host_has_profile_pic    21528 non-null  object 
 2   host_identity_verified  21528 non-null  object 
 3   has_availability        21528 non-null  object 
 4   instant_bookable        21528 non-null  object 
 5   price                   21528 non-null  object 
 6   property_type           21528 non-null  object 
 7   accommodates            21528 non-null  int64  
 8   room_type               21528 non-null  object 
 9   review_scores_rating    21528 non-null  float64
dtypes: float64(1), int64(1), object(8)
memory usage: 1.6+ MB


In [5]:
dfD['price'] = dfD['price'].replace('[\$,]', '', regex=True).astype(float)
average_price = dfD['price'].mean()
dfD['price'] = ['higher' if x >= average_price else 'lower' for x in dfD['price']]

dfD



<>:1: SyntaxWarning: invalid escape sequence '\$'
<>:1: SyntaxWarning: invalid escape sequence '\$'
C:\Users\blanf\AppData\Local\Temp\ipykernel_22196\4129812649.py:1: SyntaxWarning: invalid escape sequence '\$'
  dfD['price'] = dfD['price'].replace('[\$,]', '', regex=True).astype(float)
C:\Users\blanf\AppData\Local\Temp\ipykernel_22196\4129812649.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfD['price'] = dfD['price'].replace('[\$,]', '', regex=True).astype(float)
C:\Users\blanf\AppData\Local\Temp\ipykernel_22196\4129812649.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/

,host_is_superhost,host_has_profile_pic,host_identity_verified,has_availability,instant_bookable,price,property_type,accommodates,room_type,review_scores_rating
0,f,t,t,t,f,lower,Entire condo,2,Entire home/apt,4.90
1,f,t,t,t,f,lower,Entire condo,6,Entire home/apt,4.88
2,t,t,t,t,f,lower,Entire rental unit,4,Entire home/apt,4.91
3,f,t,t,t,f,lower,Entire rental unit,4,Entire home/apt,4.57
4,t,t,t,t,f,higher,Entire condo,6,Entire home/apt,4.94
...,...,...,...,...,...,...,...,...,...,...
21523,f,t,t,t,f,higher,Entire rental unit,4,Entire home/apt,4.92
21524,f,t,f,t,f,higher,Entire rental unit,4,Entire home/apt,4.92
21525,f,t,t,t,t,higher,Entire rental unit,3,Entire home/apt,4.92
21526,f,t,t,t,f,lower,Private room in rental unit,2,Private room,4.92


In [6]:
unique_property_types = dfD['review_scores_rating'].unique()
print(unique_property_types)


[4.9  4.88 4.91 4.57 4.94 4.83 4.58 4.89 4.73 4.97 4.72 4.78 5.   4.84
 4.92 4.71 4.8  4.77 4.54 4.87 4.7  4.82 4.74 4.33 4.6  0.   4.1  4.93
 4.67 4.   4.68 4.81 4.63 4.85 4.46 4.55 4.75 4.62 4.86 4.38 4.5  4.76
 4.69 4.66 4.64 4.52 4.79 4.44 4.29 4.95 4.96 4.07 4.49 4.43 4.61 4.39
 4.59 4.56 3.88 4.24 4.31 4.47 4.65 4.4  4.36 2.5  4.53 4.13 4.27 4.25
 4.45 3.67 4.41 4.99 4.98 4.42 4.51 4.35 3.   3.5  4.48 4.19 4.14 4.03
 4.2  4.16 4.34 2.86 4.28 4.08 4.17 4.23 4.3  4.32 3.89 4.18 4.37 4.06
 2.67 4.22 3.75 4.21 2.   4.11 3.14 3.78 3.25 4.12 4.26 1.   4.09 1.5
 3.6  3.57 3.8  4.05 3.93 3.86 3.33 4.15 3.96 3.56 3.83 3.17 3.4 ]


In [7]:
dfD['property_type'] = ['entire' if 'entire' in str(x).lower() else 'other' for x in dfD['property_type']]

print(dfD[['property_type']])


      property_type
0            entire
1            entire
2            entire
3            entire
4            entire
...             ...
21523        entire
21524        entire
21525        entire
21526         other
21527        entire

[21528 rows x 1 columns]


C:\Users\blanf\AppData\Local\Temp\ipykernel_22196\1974490582.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfD['property_type'] = ['entire' if 'entire' in str(x).lower() else 'other' for x in dfD['property_type']]


In [8]:

average_price = dfD['accommodates'].mean()
dfD['accommodates'] = ['higher' if x >= average_price else 'lower' for x in dfD['accommodates']]

dfD

C:\Users\blanf\AppData\Local\Temp\ipykernel_22196\3798317001.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfD['accommodates'] = ['higher' if x >= average_price else 'lower' for x in dfD['accommodates']]


,host_is_superhost,host_has_profile_pic,host_identity_verified,has_availability,instant_bookable,price,property_type,accommodates,room_type,review_scores_rating
0,f,t,t,t,f,lower,entire,lower,Entire home/apt,4.90
1,f,t,t,t,f,lower,entire,higher,Entire home/apt,4.88
2,t,t,t,t,f,lower,entire,higher,Entire home/apt,4.91
3,f,t,t,t,f,lower,entire,higher,Entire home/apt,4.57
4,t,t,t,t,f,higher,entire,higher,Entire home/apt,4.94
...,...,...,...,...,...,...,...,...,...,...
21523,f,t,t,t,f,higher,entire,higher,Entire home/apt,4.92
21524,f,t,f,t,f,higher,entire,higher,Entire home/apt,4.92
21525,f,t,t,t,t,higher,entire,lower,Entire home/apt,4.92
21526,f,t,t,t,f,lower,other,lower,Private room,4.92


In [ ]:
dfD['room_type'] = ['apt' if 'entire' in str(x).lower() else 'other' for x in dfD['room_type']]

print(dfD[['room_type']])

      room_type
0        entire
1        entire
2        entire
3        entire
4        entire
...         ...
21523    entire
21524    entire
21525    entire
21526     other
21527    entire

[21528 rows x 1 columns]


C:\Users\blanf\AppData\Local\Temp\ipykernel_22196\2224808896.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfD['room_type'] = ['entire' if 'entire' in str(x).lower() else 'other' for x in dfD['room_type']]


In [10]:

average_price = dfD['review_scores_rating'].mean()
dfD['review_scores_rating'] = ['higher' if x >= average_price else 'lower' for x in dfD['review_scores_rating']]

dfD

C:\Users\blanf\AppData\Local\Temp\ipykernel_22196\627463761.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfD['review_scores_rating'] = ['higher' if x >= average_price else 'lower' for x in dfD['review_scores_rating']]


,host_is_superhost,host_has_profile_pic,host_identity_verified,has_availability,instant_bookable,price,property_type,accommodates,room_type,review_scores_rating
0,f,t,t,t,f,lower,entire,lower,entire,higher
1,f,t,t,t,f,lower,entire,higher,entire,higher
2,t,t,t,t,f,lower,entire,higher,entire,higher
3,f,t,t,t,f,lower,entire,higher,entire,lower
4,t,t,t,t,f,higher,entire,higher,entire,higher
...,...,...,...,...,...,...,...,...,...,...
21523,f,t,t,t,f,higher,entire,higher,entire,higher
21524,f,t,f,t,f,higher,entire,higher,entire,higher
21525,f,t,t,t,t,higher,entire,lower,entire,higher
21526,f,t,t,t,f,lower,other,lower,other,higher


**DATAFRAME CON VARIABLES INDEPENDIENTES LIMPIAS**

In [413]:
dfI = df[['host_response_rate', 'number_of_reviews',
             'review_scores_rating', 'review_scores_communication', 
             'availability_365', 'minimum_nights', 'maximum_nights', 
             'availability_30', 'accommodates', 'bedrooms', 
             'bathrooms', 'price', 'review_scores_cleanliness', 
             'review_scores_value', 'beds','instant_bookable']]
dfI.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21528 entries, 0 to 21527
Data columns (total 16 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   host_response_rate           21528 non-null  float64
 1   number_of_reviews            21528 non-null  int64  
 2   review_scores_rating         21528 non-null  float64
 3   review_scores_communication  21528 non-null  float64
 4   availability_365             21528 non-null  int64  
 5   minimum_nights               21528 non-null  int64  
 6   maximum_nights               21528 non-null  int64  
 7   availability_30              21528 non-null  int64  
 8   accommodates                 21528 non-null  int64  
 9   bedrooms                     21528 non-null  float64
 10  bathrooms                    21528 non-null  float64
 11  price                        21528 non-null  object 
 12  review_scores_cleanliness    21528 non-null  float64
 13  review_scores_va

In [414]:
dfI['price'] = dfI['price'].replace('[\$,]', '', regex=True).astype(float)
dfI['instant_bookable'] = dfI['instant_bookable'].apply(lambda x: 1 if x == 't' else 0)

<>:1: SyntaxWarning: invalid escape sequence '\$'
<>:1: SyntaxWarning: invalid escape sequence '\$'
C:\Users\blanf\AppData\Local\Temp\ipykernel_22196\180669046.py:1: SyntaxWarning: invalid escape sequence '\$'
  dfI['price'] = dfI['price'].replace('[\$,]', '', regex=True).astype(float)
C:\Users\blanf\AppData\Local\Temp\ipykernel_22196\180669046.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfI['price'] = dfI['price'].replace('[\$,]', '', regex=True).astype(float)
C:\Users\blanf\AppData\Local\Temp\ipykernel_22196\180669046.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pan

In [26]:
dfI.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21528 entries, 0 to 21527
Data columns (total 15 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   host_response_rate           21528 non-null  float64
 1   number_of_reviews            21528 non-null  int64  
 2   review_scores_rating         21528 non-null  float64
 3   review_scores_communication  21528 non-null  float64
 4   availability_365             21528 non-null  int64  
 5   minimum_nights               21528 non-null  int64  
 6   maximum_nights               21528 non-null  int64  
 7   availability_30              21528 non-null  int64  
 8   accommodates                 21528 non-null  int64  
 9   bedrooms                     21528 non-null  float64
 10  bathrooms                    21528 non-null  float64
 11  price                        21528 non-null  float64
 12  review_scores_cleanliness    21528 non-null  float64
 13  review_scores_va

**ANALISIS**

*DEPENDIENTE-->host_is_superhost*

In [64]:
Vars_Indep1= dfI[['host_response_rate', 'number_of_reviews', 'review_scores_rating']]
Var_Dep1= dfD['host_is_superhost']
X1= Vars_Indep1
y1= Var_Dep1
X_train1, X_test1, y_train1, y_test1 = train_test_split(X1,y1,test_size=0.3, random_state=None)
escalar1= StandardScaler()
X_train1= escalar1.fit_transform(X_train1)
X_test1= escalar1.transform(X_test1)
from sklearn.linear_model import LogisticRegression
algoritmo1= LogisticRegression()

In [65]:
algoritmo1.fit(X_train1, y_train1)

LogisticRegression()

In [66]:
y_pred1= algoritmo1.predict(X_test1)
y_pred1

array(['f', 'f', 'f', ..., 'f', 'f', 'f'], shape=(6459,), dtype=object)

In [67]:
from sklearn.metrics import confusion_matrix
matriz1= confusion_matrix(y_test1, y_pred1)
print('Matriz de Confusión:')
print(matriz1)

Matriz de Confusión:
[[5647   56]
 [ 663   93]]


In [68]:
from sklearn.metrics import precision_score
precision1 = precision_score(y_test1,y_pred1,average="binary", pos_label="t")
print("Precisión del modelo:")
print(precision1)

Precisión del modelo:
0.6241610738255033


In [69]:
from sklearn.metrics import accuracy_score
exactitud1= accuracy_score(y_test1, y_pred1)
print('Exactitud del modelo:')
print(exactitud1)

Exactitud del modelo:
0.8886824585849202


In [70]:
from sklearn.metrics import recall_score
sensibilidad1= recall_score(y_test1, y_pred1, average="binary",pos_label="f")
print('Sensibilidad del modelo:')
print(sensibilidad1)

Sensibilidad del modelo:
0.990180606698229


*DEPENDIENTE-->host_has_profile_pic*

In [226]:
Vars_Indep2= dfI[['host_response_rate', 'number_of_reviews', 'review_scores_communication']]
Var_Dep2= dfD['host_has_profile_pic']
X2= Vars_Indep2
y2= Var_Dep2
X_train2, X_test2, y_train2, y_test2 = train_test_split(X2,y2,test_size=0.3, random_state=None)
escalar2= StandardScaler()
X_train2= escalar2.fit_transform(X_train2)
X_test2= escalar2.transform(X_test2)
from sklearn.linear_model import LogisticRegression
algoritmo2= LogisticRegression()

In [230]:
algoritmo2.fit(X_train2, y_train2)

LogisticRegression()

In [231]:
y_pred2= algoritmo2.predict(X_test2)
y_pred2

array(['t', 't', 't', ..., 't', 't', 't'], shape=(6459,), dtype=object)

In [232]:
from sklearn.metrics import confusion_matrix
matriz2= confusion_matrix(y_test2, y_pred2)
print('Matriz de Confusión:')
print(matriz2)

Matriz de Confusión:
[[   0  186]
 [   0 6273]]


In [123]:
from sklearn.metrics import precision_score
precision1 = precision_score(y_test1,y_pred1,average="binary", pos_label="t")
print("Precisión del modelo:")
print(precision1)

Precisión del modelo:
0.6241610738255033


In [125]:
from sklearn.metrics import accuracy_score
exactitud1= accuracy_score(y_test1, y_pred1)
print('Exactitud del modelo:')
print(exactitud1)

Exactitud del modelo:
0.8886824585849202


In [233]:
from sklearn.metrics import recall_score
sensibilidad1= recall_score(y_test1, y_pred1, average="binary",pos_label="t")
print('Sensibilidad del modelo:')
print(sensibilidad1)

Sensibilidad del modelo:
0.12301587301587301


*DEPENDIENTE-->host_identity_verified*

In [209]:
Vars_Indep3= dfI[['host_response_rate', 'number_of_reviews', 'availability_365']]
Var_Dep3= dfD['host_identity_verified']
X3= Vars_Indep3
y3= Var_Dep3
X_train3, X_test3, y_train3, y_test3= train_test_split(X3,y3,test_size=0.3, random_state=None)
escalar3= StandardScaler()
X_train3= escalar3.fit_transform(X_train3)
X_test3= escalar3.transform(X_test3)
from sklearn.linear_model import LogisticRegression
algoritmo3= LogisticRegression()

In [210]:
algoritmo3.fit(X_train3, y_train3)

LogisticRegression()

In [211]:
y_pred3= algoritmo3.predict(X_test3)
y_pred3

array(['t', 't', 't', ..., 't', 't', 't'], shape=(6459,), dtype=object)

In [212]:
print("Valores únicos de y_test3:", set(y_test3))
print("Valores únicos de y_pred3:", set(y_pred3))


Valores únicos de y_test3: {'t', 'f'}
Valores únicos de y_pred3: {'t'}


In [213]:
from sklearn.metrics import confusion_matrix
matriz3= confusion_matrix(y_test3, y_pred3)
print('Matriz de Confusión:')
print(matriz3)

Matriz de Confusión:
[[   0  775]
 [   0 5684]]


In [240]:
from sklearn.metrics import precision_score
precision3 = precision_score(y_test3,y_pred3,average="binary", pos_label="t")
print("Precisión del modelo:")
print(precision3)

Precisión del modelo:
0.8800123858182382


In [239]:
from sklearn.metrics import accuracy_score
exactitud3= accuracy_score(y_test3, y_pred3)
print('Exactitud del modelo:')
print(exactitud3)

Exactitud del modelo:
0.8800123858182382


In [236]:
from sklearn.metrics import recall_score
sensibilidad3= recall_score(y_test3, y_pred3, average="binary",pos_label="t")
print('Sensibilidad del modelo:')
print(sensibilidad3)

Sensibilidad del modelo:
1.0


*DEPENDIENTE-->has_availability*

In [376]:
Vars_Indep4= dfI[['minimum_nights', 'maximum_nights', 'availability_30', 'availability_365']]
Var_Dep4= dfD['has_availability']
X4= Vars_Indep4
y4= Var_Dep4
X_train4, X_test4, y_train4, y_test4= train_test_split(X4,y4,test_size=0.3, random_state=None)
escalar4= StandardScaler()
X_train4= escalar4.fit_transform(X_train4)
X_test4= escalar4.transform(X_test4)
from sklearn.linear_model import LogisticRegression
algoritmo4= LogisticRegression()

In [377]:
algoritmo4.fit(X_train4, y_train4)

LogisticRegression()

In [379]:
y_pred4= algoritmo4.predict(X_test4)
y_pred4

array(['t', 't', 't', ..., 't', 't', 't'], shape=(6459,), dtype=object)

In [380]:
print("Valores únicos de y_test3:", set(y_test4))
print("Valores únicos de y_pred3:", set(y_pred4))


Valores únicos de y_test3: {'t', 'f'}
Valores únicos de y_pred3: {'t', 'f'}


In [381]:
from sklearn.metrics import confusion_matrix
matriz4= confusion_matrix(y_test4, y_pred4)
print('Matriz de Confusión:')
print(matriz4)

Matriz de Confusión:
[[   0  126]
 [   1 6332]]


In [382]:
from sklearn.metrics import precision_score
precision4 = precision_score(y_test4,y_pred4,average="binary", pos_label="t")
print("Precisión del modelo:")
print(precision4)

Precisión del modelo:
0.9804893155775782


In [383]:
from sklearn.metrics import accuracy_score
exactitud4= accuracy_score(y_test4, y_pred4)
print('Exactitud del modelo:')
print(exactitud4)

Exactitud del modelo:
0.9803375135469887


In [387]:
from sklearn.metrics import recall_score
sensibilidad4= recall_score(y_test4, y_pred4, average="binary",pos_label="t")
print('Sensibilidad del modelo:')
print(sensibilidad4)

Sensibilidad del modelo:
0.9998420969524712


*DEPENDIENTE-->instant_bookable*

In [388]:
Vars_Indep1= dfI[['host_response_rate', 'number_of_reviews', 'review_scores_rating']]
Var_Dep1= dfD['instant_bookable']
X1= Vars_Indep1
y1= Var_Dep1
X_train1, X_test1, y_train1, y_test1 = train_test_split(X1,y1,test_size=0.3, random_state=None)
escalar1= StandardScaler()
X_train1= escalar1.fit_transform(X_train1)
X_test1= escalar1.transform(X_test1)
from sklearn.linear_model import LogisticRegression
algoritmo1= LogisticRegression()

In [389]:
algoritmo1.fit(X_train1, y_train1)

LogisticRegression()

In [390]:
y_pred1= algoritmo1.predict(X_test1)
y_pred1

array(['f', 'f', 'f', ..., 'f', 'f', 'f'], shape=(6459,), dtype=object)

In [391]:
from sklearn.metrics import confusion_matrix
matriz1= confusion_matrix(y_test1, y_pred1)
print('Matriz de Confusión:')
print(matriz1)

Matriz de Confusión:
[[5866   13]
 [ 570   10]]


In [392]:
from sklearn.metrics import precision_score
precision1 = precision_score(y_test1,y_pred1,average="binary", pos_label="t")
print("Precisión del modelo:")
print(precision1)

Precisión del modelo:
0.43478260869565216


In [393]:
from sklearn.metrics import accuracy_score
exactitud1= accuracy_score(y_test1, y_pred1)
print('Exactitud del modelo:')
print(exactitud1)

Exactitud del modelo:
0.9097383495897198


In [394]:
from sklearn.metrics import recall_score
sensibilidad1= recall_score(y_test1, y_pred1, average="binary",pos_label="f")
print('Sensibilidad del modelo:')
print(sensibilidad1)

Sensibilidad del modelo:
0.9977887395815614


*DEPENDIENTE-->price*

In [395]:
Vars_Indep1= dfI[['accommodates', 'bedrooms', 'bathrooms', 'number_of_reviews', 'review_scores_rating']]
Var_Dep1= dfD['price']
X1= Vars_Indep1
y1= Var_Dep1
X_train1, X_test1, y_train1, y_test1 = train_test_split(X1,y1,test_size=0.3, random_state=None)
escalar1= StandardScaler()
X_train1= escalar1.fit_transform(X_train1)
X_test1= escalar1.transform(X_test1)
from sklearn.linear_model import LogisticRegression
algoritmo1= LogisticRegression()

In [396]:
algoritmo1.fit(X_train1, y_train1)

LogisticRegression()

In [397]:
y_pred1= algoritmo1.predict(X_test1)
y_pred1

array(['higher', 'lower', 'higher', ..., 'lower', 'higher', 'lower'],
      shape=(6459,), dtype=object)

In [398]:
from sklearn.metrics import confusion_matrix
matriz1= confusion_matrix(y_test1, y_pred1)
print('Matriz de Confusión:')
print(matriz1)

Matriz de Confusión:
[[1005 1994]
 [ 727 2733]]


In [400]:
from sklearn.metrics import precision_score
precision1 = precision_score(y_test1,y_pred1,average="binary", pos_label="higher")
print("Precisión del modelo:")
print(precision1)

Precisión del modelo:
0.5802540415704388


In [401]:
from sklearn.metrics import accuracy_score
exactitud1= accuracy_score(y_test1, y_pred1)
print('Exactitud del modelo:')
print(exactitud1)

Exactitud del modelo:
0.5787273571760334


In [403]:
from sklearn.metrics import recall_score
sensibilidad1= recall_score(y_test1, y_pred1, average="binary",pos_label="lower")
print('Sensibilidad del modelo:')
print(sensibilidad1)

Sensibilidad del modelo:
0.7898843930635838


*DEPENDIENTE-->property_type*

In [404]:
Vars_Indep1= dfI[['accommodates', 'bedrooms', 'price', 'review_scores_rating']]
Var_Dep1= dfD['property_type']
X1= Vars_Indep1
y1= Var_Dep1
X_train1, X_test1, y_train1, y_test1 = train_test_split(X1,y1,test_size=0.3, random_state=None)
escalar1= StandardScaler()
X_train1= escalar1.fit_transform(X_train1)
X_test1= escalar1.transform(X_test1)
from sklearn.linear_model import LogisticRegression
algoritmo1= LogisticRegression()

In [405]:
algoritmo1.fit(X_train1, y_train1)

LogisticRegression()

In [406]:
y_pred1= algoritmo1.predict(X_test1)
y_pred1

array(['entire', 'entire', 'entire', ..., 'entire', 'entire', 'entire'],
      shape=(6459,), dtype=object)

In [407]:
from sklearn.metrics import confusion_matrix
matriz1= confusion_matrix(y_test1, y_pred1)
print('Matriz de Confusión:')
print(matriz1)

Matriz de Confusión:
[[5859    9]
 [ 486  105]]


In [409]:
from sklearn.metrics import precision_score
precision1 = precision_score(y_test1,y_pred1,average="binary", pos_label="entire")
print("Precisión del modelo:")
print(precision1)

Precisión del modelo:
0.9234042553191489


In [410]:
from sklearn.metrics import accuracy_score
exactitud1= accuracy_score(y_test1, y_pred1)
print('Exactitud del modelo:')
print(exactitud1)

Exactitud del modelo:
0.9233627496516489


In [411]:
from sklearn.metrics import recall_score
sensibilidad1= recall_score(y_test1, y_pred1, average="binary",pos_label="other")
print('Sensibilidad del modelo:')
print(sensibilidad1)

Sensibilidad del modelo:
0.17766497461928935


*DEPENDIENTE-->accommodates*

In [415]:
Vars_Indep1= dfI[['bedrooms', 'bathrooms', 'beds', 'price', 'number_of_reviews']]
Var_Dep1= dfD['accommodates']
X1= Vars_Indep1
y1= Var_Dep1
X_train1, X_test1, y_train1, y_test1 = train_test_split(X1,y1,test_size=0.3, random_state=None)
escalar1= StandardScaler()
X_train1= escalar1.fit_transform(X_train1)
X_test1= escalar1.transform(X_test1)
from sklearn.linear_model import LogisticRegression
algoritmo1= LogisticRegression()

In [416]:
algoritmo1.fit(X_train1, y_train1)

LogisticRegression()

In [417]:
y_pred1= algoritmo1.predict(X_test1)
y_pred1

array(['lower', 'higher', 'higher', ..., 'lower', 'lower', 'lower'],
      shape=(6459,), dtype=object)

In [418]:
from sklearn.metrics import confusion_matrix
matriz1= confusion_matrix(y_test1, y_pred1)
print('Matriz de Confusión:')
print(matriz1)

Matriz de Confusión:
[[2252  490]
 [ 342 3375]]


In [420]:
from sklearn.metrics import precision_score
precision1 = precision_score(y_test1,y_pred1,average="binary", pos_label="higher")
print("Precisión del modelo:")
print(precision1)

Precisión del modelo:
0.8681572860447185


In [421]:
from sklearn.metrics import accuracy_score
exactitud1= accuracy_score(y_test1, y_pred1)
print('Exactitud del modelo:')
print(exactitud1)

Exactitud del modelo:
0.8711874903235794


In [422]:
from sklearn.metrics import recall_score
sensibilidad1= recall_score(y_test1, y_pred1, average="binary",pos_label="lower")
print('Sensibilidad del modelo:')
print(sensibilidad1)

Sensibilidad del modelo:
0.9079903147699758


*DEPENDIENTE-->instant_bookable*

In [423]:
Vars_Indep1= dfI[['accommodates', 'bedrooms', 'price', 'instant_bookable']]
Var_Dep1= dfD['room_type']
X1= Vars_Indep1
y1= Var_Dep1
X_train1, X_test1, y_train1, y_test1 = train_test_split(X1,y1,test_size=0.3, random_state=None)
escalar1= StandardScaler()
X_train1= escalar1.fit_transform(X_train1)
X_test1= escalar1.transform(X_test1)
from sklearn.linear_model import LogisticRegression
algoritmo1= LogisticRegression()

In [424]:
algoritmo1.fit(X_train1, y_train1)

LogisticRegression()

In [425]:
y_pred1= algoritmo1.predict(X_test1)
y_pred1

array(['entire', 'entire', 'entire', ..., 'entire', 'entire', 'entire'],
      shape=(6459,), dtype=object)

In [426]:
from sklearn.metrics import confusion_matrix
matriz1= confusion_matrix(y_test1, y_pred1)
print('Matriz de Confusión:')
print(matriz1)

Matriz de Confusión:
[[5885   11]
 [ 443  120]]


In [428]:
from sklearn.metrics import precision_score
precision1 = precision_score(y_test1,y_pred1,average="binary", pos_label="entire")
print("Precisión del modelo:")
print(precision1)

Precisión del modelo:
0.9299936788874842


In [429]:
from sklearn.metrics import accuracy_score
exactitud1= accuracy_score(y_test1, y_pred1)
print('Exactitud del modelo:')
print(exactitud1)

Exactitud del modelo:
0.929710481498684


In [430]:
from sklearn.metrics import recall_score
sensibilidad1= recall_score(y_test1, y_pred1, average="binary",pos_label="other")
print('Sensibilidad del modelo:')
print(sensibilidad1)

Sensibilidad del modelo:
0.21314387211367672


*DEPENDIENTE-->review_scores_rating*

In [431]:
Vars_Indep1= dfI[['number_of_reviews', 'review_scores_cleanliness', 'review_scores_communication', 'review_scores_value']]
Var_Dep1= dfD['instant_bookable']
X1= Vars_Indep1
y1= Var_Dep1
X_train1, X_test1, y_train1, y_test1 = train_test_split(X1,y1,test_size=0.3, random_state=None)
escalar1= StandardScaler()
X_train1= escalar1.fit_transform(X_train1)
X_test1= escalar1.transform(X_test1)
from sklearn.linear_model import LogisticRegression
algoritmo1= LogisticRegression()

In [432]:
algoritmo1.fit(X_train1, y_train1)

LogisticRegression()

In [433]:
y_pred1= algoritmo1.predict(X_test1)
y_pred1

array(['f', 'f', 'f', ..., 'f', 'f', 'f'], shape=(6459,), dtype=object)

In [434]:
from sklearn.metrics import confusion_matrix
matriz1= confusion_matrix(y_test1, y_pred1)
print('Matriz de Confusión:')
print(matriz1)

Matriz de Confusión:
[[5863   13]
 [ 571   12]]


In [435]:
from sklearn.metrics import precision_score
precision1 = precision_score(y_test1,y_pred1,average="binary", pos_label="t")
print("Precisión del modelo:")
print(precision1)

Precisión del modelo:
0.48


In [436]:
from sklearn.metrics import accuracy_score
exactitud1= accuracy_score(y_test1, y_pred1)
print('Exactitud del modelo:')
print(exactitud1)

Exactitud del modelo:
0.9095835268617433


In [437]:
from sklearn.metrics import recall_score
sensibilidad1= recall_score(y_test1, y_pred1, average="binary",pos_label="f")
print('Sensibilidad del modelo:')
print(sensibilidad1)

Sensibilidad del modelo:
0.9977876106194691
